# Integrated Academic Information (2021–2023)

* Integrated semester-level academic data for the three-year period from 2021 to 2023.
* Added labels for prediction horizons of $t=1, 2, 3$.

### Setup

In [77]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import pandas as pd
import os

## Data Loading

In [ ]:
# [CONFIG REQUIRED] Define column names matching your raw data schema
col_names = ['year', 'semester', 'grade', 'state', 'change_date', 'return_date', 'sex', 'college', 'dept', 
             'nation', 'adm_unit', 'adm_unit_type', 'capa', 'gpa_major', 'n_major', 'gpa_core', 'n_core', 'gpa_tot', 'credits',
             'gpa_last_seme', 'credits_last_seme', 'credits_last_tot', 'n_semesters', 'years_since', 'sno_hashed']

# [CONFIG REQUIRED] Update file paths and loading logic for your institution's data
# Currently configured to load 6 semester snapshots from a single Excel file.
data_211 = pd.read_excel("datasets/academic_data_21-23.xlsx", sheet_name="2021-1", names=col_names)
data_212 = pd.read_excel("datasets/academic_data_21-23.xlsx", sheet_name="2021-2", names=col_names)
data_221 = pd.read_excel("datasets/academic_data_21-23.xlsx", sheet_name="2022-1", names=col_names)
data_222 = pd.read_excel("datasets/academic_data_21-23.xlsx", sheet_name="2022-2", names=col_names)
data_231 = pd.read_excel("datasets/academic_data_21-23.xlsx", sheet_name="2023-1", names=col_names)
data_232 = pd.read_excel("datasets/academic_data_21-23.xlsx", sheet_name="2023-2", names=col_names)

df = pd.concat([data_211, data_212, data_221, data_222, data_231, data_232], axis=0)
df['state_next_1'] = df['state_next_2'] =  df['state_next_3'] = np.nan
df.reset_index(inplace=True)

In [79]:
df['change_date'] = pd.to_datetime(df['change_date'], format='%Y%m%d')
df['return_date'] = pd.to_datetime(df['return_date'], format='%Y%m%d')
#data_21.astype({'n_major': 'int64', 'n_core': 'int64', 'credits': 'int64', 'credits_last_seme': 'int64', 
#                'credits_last_tot': 'int64', 'n_semesters': 'int64'})

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81102 entries, 0 to 81101
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   index              81102 non-null  int64         
 1   year               81102 non-null  int64         
 2   semester           81102 non-null  int64         
 3   grade              81102 non-null  int64         
 4   state              81102 non-null  object        
 5   change_date        81102 non-null  datetime64[ns]
 6   return_date        12226 non-null  datetime64[ns]
 7   sex                81102 non-null  object        
 8   college            81102 non-null  object        
 9   dept               81102 non-null  object        
 10  nation             81102 non-null  object        
 11  adm_unit           80844 non-null  object        
 12  adm_unit_type      80844 non-null  object        
 13  capa               81102 non-null  object        
 14  gpa_ma

In [ ]:
# [CONFIG REQUIRED] Map your institution's status labels to standard codes
# 0: Enrolled, 1: Graduated/Completed, 2: Dropped Out, 3: On leave
state_code = {
    '재학': 0, # enrolled
    '졸업': 1, '수료': 1, '학사학위취득유예': 1, # graduated
    '제적': 2, # dropped out
    '휴학': 3, # on leave
}
df['state_code'] = df['state'].map(state_code)


In [82]:
sex_code = {
    '남': 'M', '여': 'F'
}
df['sex'] = df['sex'].map(sex_code)

In [ ]:
# [CONFIG REQUIRED] Update college/department mappings
college_code = {
    '건강보건대학': 'HT',
    '공과대학': 'EG',
    '글로벌비즈니스대학': 'BZ',
    '소프트웨어융합대학': 'SW',
    'SW융합대학': 'SW',
    '신학대학': 'TH',
    '예술체육대학': 'AT',
    '인문사회대학': 'HS'
}
df['college_code'] = df['college'].map(college_code)

In [ ]:
# [CONFIG REQUIRED] Update admission unit/type mappings
adm_unit_code = {
    '수시1차': 0,
    '수시2차': 0,
    '수시모집': 0,
    '정시다군': 1,
    '정시나군': 1,
    '외국인특례': 2,
    '외국인특례-2': 2,
    '편입외국인특례-2': 2,
    '편입외국인특례': 2,
    '추가1차': 3,
    '추가2차': 3,
    '추가3차': 3,
    '추가4차': 3,
    '편입모집': 4,
    '편입모집2차': 4,
    '서남대특별편입학': 4,
    '한중대특별편입학': 4,
    '서남대특별편입학2차': 4,
    '한중대특별편입학2차': 4
}
df['adm_unit_code'] = df['adm_unit'].map(adm_unit_code)

In [85]:
nation_code = {'한국': 0, '일본': 1, '베트남': 2, '중국': 3, '우즈베키스탄': 4}
df['nation_code'] = df['nation'].map(nation_code)
df.loc[df['nation_code'].isna(), 'nation_code'] = 5
df['nation_code'].unique()

array([0., 1., 2., 3., 5., 4.])

In [86]:
in_capa = (df['capa'] == '정원내')
df.loc[in_capa, 'in_capa'] = True
df.loc[~in_capa, 'in_capa'] = False

In [87]:
df = df.drop(columns=['change_date', 'return_date', 'dept', 'adm_unit_type'])

In [88]:
data = {'21-1': data_211, '21-2': data_212,
        '22-1': data_221, '22-2': data_222,
        '23-1': data_231, '23-2': data_232}


In [89]:

for index, row in df[['year', 'semester', 'sno_hashed']].iterrows():
    year = row['year']; seme = row['semester']; sno_hashed = row['sno_hashed']
    if (seme == 1):
        year_next_1 = year; seme_next_1 = 2
        year_next_2 = year + 1; seme_next_2 = 1
        year_next_3 = year + 1; seme_next_3 = 2
    else:
        year_next_1 = year + 1; seme_next_1 = 1
        year_next_2 = year_next_1; seme_next_2 = 2 
        year_next_3 = year + 2; smem_next_3 = 1

    if (year_next_1 == 2024):
        continue

    # Get the state of the first next semester
    book = data[str(year_next_1 % 2000) + '-' + str(seme_next_1)]
    if (book['sno_hashed'] == sno_hashed).any(): # is any true ?
        df.loc[index, 'state_next_1'] = state_code[book.loc[book['sno_hashed'] == sno_hashed, 'state'].values[0]]

    if (year_next_2 == 2024):
        continue

    # Get the state of the second next semester
    book = data[str(year_next_2 % 2000) + '-' + str(seme_next_2)]
    if (book['sno_hashed'] == sno_hashed).any():
        df.loc[index, 'state_next_2'] = state_code[book.loc[book['sno_hashed'] == sno_hashed, 'state'].values[0]]

    if (year_next_3 == 2024):
        continue

    # Get the state of the third next semester
    book = data[str(year_next_3 % 2000) + '-' + str(seme_next_3)]
    if (book['sno_hashed'] == sno_hashed).any():
        df.loc[index, 'state_next_3'] = state_code[book.loc[book['sno_hashed'] == sno_hashed, 'state'].values[0]]



Adding the ‘leave’ column indicating whether the student was on leave in that semester.

In [90]:
leave = (df['state_code'].isin([3]))
df.loc[~leave.values, 'leave'] = False
df.loc[leave.values, 'leave'] = True

Pre-propessing missing values

In [91]:
df.drop(['gpa_tot', 'credits', 'gpa_major', 'gpa_core', 'n_major', 'n_core', 
              'sno_hashed', 'state', 'college', 'nation', 'adm_unit', 'capa'], axis=1, inplace=True)
df.dropna(subset=['gpa_last_seme', 'credits_last_seme', 'credits_last_tot'], inplace=True)

* Handling missing values in the admission type variable('adm_unit_code')

In [92]:
df.loc[df['nation_code'] != 0, 'adm_unit_code'] = df.loc[df['nation_code'] != 0, 'adm_unit_code'].fillna(2) # for foreign students
df.loc[df['nation_code'] == 0, 'adm_unit_code'] = df.loc[df['nation_code'] == 0, 'adm_unit_code'].fillna(0) # for domestic students

Type setting and column renaming

In [93]:
df= df.astype({'adm_unit_code': 'int64', 'nation_code': 'int64', 'n_semesters': 'int64'})
df.rename(columns={'credits_last_tot': 'credits_tot', 'n_semesters': 'n_seme', 'state_code': 'state_now',
                        'adm_unit_code': 'adm_unit', 'nation_code': 'nation', 'college_code': 'college'}, inplace=True)

Labeling for stratified sampling

In [94]:
is_drop = df['state_now'].isin([2]) # 2: dropout
df.loc[is_drop, 'is_drop'] = 1
df.loc[~is_drop, 'is_drop'] = 0

In [95]:
df.reset_index(inplace=True, drop=True)

In [96]:
df.drop(['index'], axis=1, inplace=True)

In [97]:
df.to_csv('datasets/aca_21-23.csv', index=False, na_rep='NULL', encoding='utf-8-sig')